In [ ]:
!pip install pytrends


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import datetime # Import to potentially help with unique filenames

# 1. Initialization (Using simplified TrendReq to avoid past errors)
try:
    pytrends = TrendReq(hl='en-US', tz=360) 
except Exception as e:
    print(f"Error during TrendReq initialization: {e}")
    exit()

# Define parameters
KW_LIST = ['Bitcoin', 'Ethereum']
TIMEFRAME = '2020-01-01 2024-12-31' 
GEO = '' 

# Define the filename based on the data
current_date = datetime.date.today().strftime("%Y%m%d")
CSV_FILENAME = f"google_trends_daily_{current_date}.csv"

print(f"Fetching Google Trends data for {KW_LIST} from {TIMEFRAME}...")

try:
    pytrends.build_payload(
        kw_list=KW_LIST, 
        cat=0, 
        timeframe=TIMEFRAME, 
        geo=GEO
    )
    
    # Get weekly data for long timeframes
    df_trends_weekly = pytrends.interest_over_time()
    
    # 2. Clean and Rename Columns
    if 'isPartial' in df_trends_weekly.columns:
        df_trends_weekly = df_trends_weekly.drop(columns=['isPartial'])
        
    df_trends_weekly.index = df_trends_weekly.index.normalize()
    df_trends_weekly.columns = ['GTrends_BTC', 'GTrends_ETH']
    
    # 3. Convert Weekly Data to a Daily Index (Forward Fill)
    df_trends_daily = df_trends_weekly.resample('D').ffill()
    df_trends_daily = df_trends_daily.loc[TIMEFRAME.split()[0]:TIMEFRAME.split()[1]]

    # 4. Save the DataFrame to a CSV file
    # index=True ensures the Date column is saved as the first column
    df_trends_daily.to_csv(CSV_FILENAME, index=True) 
    
    print("\n----------------------------------------------------")
    print(f"Data saved to: {CSV_FILENAME}")
    print(f"First 5 rows:\n{df_trends_daily.head().to_markdown()}")
    print("----------------------------------------------------")
    
    # Final dataframe for use in your script (if needed)
    df_trends_for_joining = df_trends_daily
    
except Exception as e:
    print(f"An error occurred while fetching Google Trends data: {e}")